<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_8_Sugar_and_Spice_Step_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 7 and intro

Script Notes: 

In lesson 7 we introduced traders into our model. Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again or you can mount your drive and not have to load it every time   

Now we want to start making our agents do things. In this lesson we are going to add the sugar and spice step functions.

The way Mesa is developed following ABM convention is that it will look for step functions for each agent in the schedule. So users must call the function that causes each agent to take action **step**.


Review each class and model set up

In [1]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 6.6 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 


## Part 1: Add sugar step

**Part 1**

-----
First we add a function to the class called step


Again there is several ways to code this, using a compact line of code we are going to exploit python min function and make a list that compares the max sugar the cell should have with the amount it does have and go with whivhver the minimum is. This ensure we never go above the max sugar but allows the sugar to grow back one at a time. 

Now that this is complete we need the model to actually call the function so steps can proceed, so lets move down to the main model class. 



In [14]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      # print(self.unique_id, self.amount, self.max_sugar) #delete for part 4
      

In [20]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn.
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
      print(self.unique_id, self.amount, self.max_spice) #delete for part 4


       

In [4]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 

      

## Add step function and run model

Now we want to add step function for the model. However as discussed in lesson two this model has a unique set up  in that is does a staged activation - sugar grows, spice grows, then agents do each one of there steps, move, consume, trade and activation by type sugar, spice, traders. Mesa has staged activation and random activation by type but not a combined approach. However, the great think about using python is we can easily set this up ourselves so Mesa really becomes a menu of options that users can cusotmize as they see fit.   

The result is our step function will be slighlty more complex than what you can see in other mesa examples. 

**Part 2**

First thing we add the step function for the model and then since we are using activation by type which is just a dictionary of the different types of agents we will call the Sugar class and interates through its values

**Part 3**

Now we want to iterate through some number of steps. There are different ways to do this, for exmaple call step from our model object we created, but in this case we are going to create function called run model in or main sugarscapeg1mt class

I will add a key word argument called step count and then create a loop for that number of steps, I will defualt to 1000 as that is what is in the book but then I will test to make sure the sugar agent is doing what we want for 5 steps.

So first I will add a print statement then and call the run_model function

In [22]:
class SugarscapeG1mt(mesa.Model):

  #part 0
  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min =1,
               metabolism_max = 5, capability_min=1, capability_max=5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population #part 0
    self.endowment_min = endowment_min #part3
    self.endowment_max = endowment_max #part3
    self.capability_min = capability_min #part4
    self.capability_max = capability_max #part4


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    sugar_distribution = np.genfromtxt("sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    # part 1
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) #part 2a
      y = self.random.randrange(self.height) #part 2a  
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) #part 3
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) #part 3
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.capability_min, self.capability_max)
      metabolism_spice = self.random.uniform(self.capability_min, self.capability_max)
      #add vision
      vision =int(self.random.uniform(self.capability_min, self.capability_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self): #part 2
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()

    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()

    self.schedule.time +=1
    self.schedule.steps += 1
    

  def run_model(self, step_count=1000):

    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

**Part 3**

Now I will add the run model function and as we can see we get agents with the same values for max_sugar and amount as expected. 

Another way to do this would be to run a loop here for example if we comment out the run_model and call the step function, we see the same result. 


**Part 4**

However, there is a bug, since we are not calling mesa schedule.step function which would look like this there is bug as our steps and time are not advancing. We can see this with a print statement. first lets get rid our our sugar print statement call our time and steps and see what there values are.

As you can see they remain 0. This will have implications for our data collection later on so let's fix this now. In the step functin we will simply update the schedule values for time and steps. 

If you are new to ABMs this may see a little annoying but as you make more complex models this ability to shape the details becomes critical and highly desirable, thanks to the dynamics of complex system Mesa will never be able to account for all variation, so our goal is to provide the major items that you can customize as you need to.

Now that we have tested and know it works we can add the step function for the spice agent, which will be the same and call it from the model step function. We update those cells and test and viola it works. 

This concludes session 8, and next we will starting adding behaviors to the traders, specifically movement. 

In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=5) #part 3

'''
for i in range(5):
  model.step()
'''